In [1]:
import os

try:
    # Running as normal Python script inside src/
    this_file = os.path.abspath(__file__)
    src_root = os.path.dirname(this_file)                        # EMOTION-PRED/src
    project_root = os.path.dirname(src_root)                    # EMOTION-PRED/
except NameError:
    # Running inside Jupyter (likely src/notebooks or src/)
    cwd = os.getcwd()

    # If running inside src/notebooks → go up one level
    if cwd.endswith("notebooks"):
        src_root = os.path.abspath(os.path.join(cwd, ".."))
        project_root = os.path.dirname(src_root)
    else:
        # Running from project root directly
        project_root = cwd
        src_root = os.path.join(project_root, "src")

# Final unified paths
results_root = os.path.join(src_root, "results")
data_root = os.path.join(src_root, "data","MAMS-ACSA","raw","data_jsonl")
print(f"📂 Project root: {project_root}"
      f"\n📂 Source root: {src_root}"
      f"\n📂 Results root: {results_root}"
      f"\n📂 Data root: {data_root}")
# 3 — JSONL files
TRAIN_JSONL = os.path.join(data_root, "train.jsonl")
VAL_JSONL   = os.path.join(data_root, "val.jsonl")
TEST_JSONL  = os.path.join(data_root, "test.jsonl")
SAMPLE_JSONL = os.path.join(data_root, "sample.jsonl")
print("Using dataset directory:", data_root)



📂 Project root: /Users/hd/Desktop/EMOTION-PRED
📂 Source root: /Users/hd/Desktop/EMOTION-PRED/src
📂 Results root: /Users/hd/Desktop/EMOTION-PRED/src/results
📂 Data root: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl
Using dataset directory: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl


In [2]:
# # ============================================================
# # VALIDATION SCRIPT FOR 300 GOLD ANNOTATIONS
# # Evaluates:
# #  - Emotion Macro-F1 / Micro-F1
# #  - Aspect Macro-F1
# #  - Polarity Macro-F1
# #  - Full ABSA triple accuracy
# # ============================================================

# import json
# import pandas as pd
# from sklearn.metrics import f1_score, classification_report, precision_recall_fscore_support

# # ------------------------------------------------------------
# # PATHS - UPDATE ONLY THESE IF NEEDED
# # ------------------------------------------------------------
# GOLD_PATH = os.path.join(data_root, "sample_06_12_2025_6pm_annotated.jsonl")

# # Option A → Emotion-only model output
# PRED_PATH = "output/gemini_annotated.jsonl"

# # Option B → Full aspect+polarity+emotion prediction
# # PRED_PATH = "output/gemini_full_absa.jsonl"

# # ------------------------------------------------------------
# # LOAD FILES
# # ------------------------------------------------------------
# def load_jsonl(path):
#     rows = []
#     with open(path, "r", encoding="utf-8") as f:
#         for line in f:
#             rows.append(json.loads(line))
#     return rows

# gold = load_jsonl(GOLD_PATH)
# pred = load_jsonl(PRED_PATH)

# assert len(gold) == len(pred), f"Row mismatch: gold={len(gold)}, pred={len(pred)}"
# print(f"Loaded {len(gold)} rows ✓")

# # ------------------------------------------------------------
# # EXPLODE INTO FLAT FORM
# # ------------------------------------------------------------
# def explode(data):
#     rows = []
#     for i, row in enumerate(data):
#         for item in row["output"]:
#             rows.append({
#                 "id": i,
#                 "aspect": item.get("aspect"),
#                 "polarity": item.get("polarity"),
#                 "emotion": item.get("emotion")
#             })
#     return pd.DataFrame(rows)

# gold_df = explode(gold)
# pred_df = explode(pred)

# print("Gold rows:", len(gold_df))
# print("Pred rows:", len(pred_df))

# # ------------------------------------------------------------
# # MERGE GOLD + PRED ON (ID, ASPECT, POLARITY)
# # ------------------------------------------------------------
# merged = gold_df.merge(
#     pred_df,
#     on=["id", "aspect", "polarity"],
#     suffixes=("_gold", "_pred"),
#     how="outer"
# )

# print("Merged rows:", len(merged))

# # Drop rows where emotion missing in gold or pred
# valid = merged.dropna(subset=["emotion_gold", "emotion_pred"])

# # ------------------------------------------------------------
# # EMOTION F1
# # ------------------------------------------------------------
# y_true = valid["emotion_gold"]
# y_pred = valid["emotion_pred"]

# print("\n==============================")
# print("EMOTION SCORES")
# print("==============================")
# print("Emotion Macro-F1:", f1_score(y_true, y_pred, average='macro'))
# print("Emotion Micro-F1:", f1_score(y_true, y_pred, average='micro'))

# print("\nClassification report:")
# print(classification_report(y_true, y_pred))

# # ------------------------------------------------------------
# # ASPECT F1 (if full ABSA predictions contain aspects)
# # ------------------------------------------------------------
# print("\n==============================")
# print("ASPECT SCORES")
# print("==============================")

# aspect_true = gold_df["aspect"]
# aspect_pred = pred_df["aspect"]

# aspect_f1 = f1_score(aspect_true, aspect_pred, average="macro")
# print("Aspect Macro-F1:", aspect_f1)

# # ------------------------------------------------------------
# # POLARITY F1
# # ------------------------------------------------------------
# print("\n==============================")
# print("POLARITY SCORES")
# print("==============================")

# pol_true = gold_df["polarity"]
# pol_pred = pred_df["polarity"]

# pol_f1 = f1_score(pol_true, pol_pred, average="macro")
# print("Polarity Macro-F1:", pol_f1)

# # ------------------------------------------------------------
# # FULL ABSA TRIPLE ACCURACY
# # aspect + polarity + emotion must match exactly
# # ------------------------------------------------------------
# print("\n==============================")
# print("FULL ABSA TRIPLE ACCURACY")
# print("==============================")

# merged["triple_gold"] = list(
#     zip(merged["aspect"], merged["polarity"], merged["emotion_gold"])
# )
# merged["triple_pred"] = list(
#     zip(merged["aspect"], merged["polarity"], merged["emotion_pred"])
# )

# merged["correct_triple"] = merged["triple_gold"] == merged["triple_pred"]
# triple_accuracy = merged["correct_triple"].mean()

# print("Full ABSA Accuracy:", triple_accuracy)

# # ------------------------------------------------------------
# # SUMMARY TABLE
# # ------------------------------------------------------------
# print("\n==============================")
# print("SUMMARY")
# print("==============================")

# print(f"""
# Emotion Macro-F1      : {f1_score(y_true, y_pred, average='macro'):.4f}
# Emotion Micro-F1      : {f1_score(y_true, y_pred, average='micro'):.4f}
# Aspect Macro-F1       : {aspect_f1:.4f}
# Polarity Macro-F1     : {pol_f1:.4f}
# Full ABSA Accuracy    : {triple_accuracy:.4f}
# """)

In [3]:
import json
import os
import pandas as pd
from sklearn.metrics import f1_score, classification_report
from collections import Counter

# -----------------------------
# Paths
# -----------------------------
GOLD_PATH = os.path.join(data_root, "cleaned_300.jsonl")
PRED_PATH = "/Users/hd/Desktop/EMOTION-PRED/src/notebooks/output/gemini_annotated_aspect_polarity_emotions_200.jsonl"

# -----------------------------
# Load JSONL
# -----------------------------
def load_jsonl(path):
    return [json.loads(line) for line in open(path, "r", encoding="utf-8")]

gold = load_jsonl(GOLD_PATH)
pred = load_jsonl(PRED_PATH)

# Map gold/pred by input text
gold_map = {row["input"]: row["output"] for row in gold}
pred_map = {row["input"]: row["output"] for row in pred}

print(f"Loaded gold rows: {len(gold_map)}")
print(f"Loaded pred rows: {len(pred_map)}")


# -----------------------------
# Triple alignment
# -----------------------------
def align_triples(gold_list, pred_list):
    """
    Ensures aligned triples for scoring:
    - If pred has MORE triples → truncate
    - If pred has FEWER triples → pad with ('none','none','none')
    """
    g = [(t["aspect"], t["polarity"], t["emotion"]) for t in gold_list]
    p = [(t["aspect"], t["polarity"], t["emotion"]) for t in pred_list]

    gold_n = len(g)
    pred_n = len(p)

    # truncate hallucinations
    if pred_n > gold_n:
        p = p[:gold_n]

    # pad missing predictions
    if pred_n < gold_n:
        pad = [("none", "none", "none")] * (gold_n - pred_n)
        p.extend(pad)

    return g, p


# -----------------------------
# Collect aligned labels
# -----------------------------
all_gold_as, all_pred_as = [], []
all_gold_pol, all_pred_pol = [], []
all_gold_emo, all_pred_emo = [], []

gold_triples_full = []
pred_triples_full = []

for text, gold_list in gold_map.items():

    pred_list = pred_map.get(text, [])

    g_aligned, p_aligned = align_triples(gold_list, pred_list)

    for (ga, gp, ge), (pa, pp, pe) in zip(g_aligned, p_aligned):
        all_gold_as.append(ga)
        all_gold_pol.append(gp)
        all_gold_emo.append(ge)

        all_pred_as.append(pa)
        all_pred_pol.append(pp)
        all_pred_emo.append(pe)

        gold_triples_full.append((ga, gp, ge))
        pred_triples_full.append((pa, pp, pe))


# -----------------------------
# Compute metrics
# -----------------------------
aspect_f1 = f1_score(all_gold_as, all_pred_as, average="macro", zero_division=0)
polarity_f1 = f1_score(all_gold_pol, all_pred_pol, average="macro", zero_division=0)
emotion_macro = f1_score(all_gold_emo, all_pred_emo, average="macro", zero_division=0)
emotion_micro = f1_score(all_gold_emo, all_pred_emo, average="micro", zero_division=0)

# Full ABSA accuracy (exact triple match)
match = sum(1 for g, p in zip(gold_triples_full, pred_triples_full) if g == p)
exact_acc = match / len(gold_triples_full)


# -----------------------------
# Pretty PRINT
# -----------------------------
print("\n==============================")
print("FINAL EVALUATION REPORT")
print("==============================\n")

print("ASPECT CLASSIFICATION")
print("------------------------------")
print(f"Macro F1: {aspect_f1:.4f}\n")

print("POLARITY CLASSIFICATION")
print("------------------------------")
print(f"Macro F1: {polarity_f1:.4f}\n")

print("EMOTION CLASSIFICATION")
print("------------------------------")
print(f"Emotion Macro-F1 : {emotion_macro:.4f}")
print(f"Emotion Micro-F1 : {emotion_micro:.4f}\n")

print("FULL ABSA TRIPLE MATCH")
print("------------------------------")
print(f"Full ABSA Accuracy: {exact_acc:.4f}\n")

print("==============================")
print("DETAILED CLASSIFICATION REPORT")
print("==============================\n")
print(classification_report(all_gold_emo, all_pred_emo, zero_division=0))


# Save optional CSV
df = pd.DataFrame({
    "Aspect F1":      [aspect_f1],
    "Polarity F1":    [polarity_f1],
    "Emotion F1":     [emotion_macro],
    "Emotion Micro":  [emotion_micro],
    "Full ABSA Acc":  [exact_acc]
})
df.to_csv("eval_results.csv", index=False)

print("\nSaved → eval_results.csv")

Loaded gold rows: 300
Loaded pred rows: 300

FINAL EVALUATION REPORT

ASPECT CLASSIFICATION
------------------------------
Macro F1: 0.4346

POLARITY CLASSIFICATION
------------------------------
Macro F1: 0.3454

EMOTION CLASSIFICATION
------------------------------
Emotion Macro-F1 : 0.2475
Emotion Micro-F1 : 0.2965

FULL ABSA TRIPLE MATCH
------------------------------
Full ABSA Accuracy: 0.2175

DETAILED CLASSIFICATION REPORT

                precision    recall  f1-score   support

    Admiration       0.49      0.37      0.42        52
     Annoyance       0.35      0.55      0.43        82
  Appreciation       0.00      0.00      0.00         0
      Approval       0.35      0.41      0.38        44
     Confusion       0.33      0.22      0.27         9
Disappointment       0.30      0.39      0.34        70
   Disapproval       0.36      0.17      0.23        24
       Disgust       0.00      0.00      0.00         0
    Excitement       0.00      0.00      0.00         2
    